In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import random as rnd
from scipy import signal
import dcnv, imp
imp.reload(dcnv)

# oasis needs some parameters
ops = {
        'tau':  1., # this is the only parameter used for deconvolution per se        
        'fs': 10.,  # sampling rate        
        'baseline': 'maximin', # baselining mode
        'win_baseline': 60., # window for maximin
        'sig_baseline': 10., # smoothing constant for gaussian filter 
        'prctile_baseline': 8.,# smoothing constant for gaussian filter        
        'neucoeff': .7,  # neuropil coefficient 
        'neumax': 1.,  # maximum neuropil coefficient (not implemented)
        'niterneu': 5, # number of iterations when the neuropil coefficient is estimated (not implemented)
      }

# simulate some data
NT = 20000
p = .01
ncells = 100
sig_noise = 0.35

sp = p - rnd.uniform(0,1,(NT,ncells))>0
kern = np.exp(-np.arange(100.)/(ops['fs'] * ops['tau']))

F = signal.convolve(np.expand_dims(kern, axis = 1), sp, mode='full')

F = F[:NT, :]      

F = F + rnd.normal(0,sig_noise,(NT,ncells))
F = np.transpose(F)

plt.imshow(F[:,:1000], aspect='auto')
plt.show()

#plt.imshow(np.transpose(F), aspect='auto')
plt.plot(F[0,:1000])
plt.show()

In [ ]:
# run deconvolution
s = dcnv.oasis(F, ops)

# compare with simulated ground truth spike trains
b = np.zeros((ncells,))
for i in range(0,ncells):
    A = np.corrcoef(sp[:,i], s[i, :20000])
    b[i] = A[0,1]
    
print('correlation with ground truth %2.4f'%b.mean())
